This note book is a modified version of one of the notebooks used in this great course from Coursera: https://www.coursera.org/learn/natural-language-processing-tensorflow


In [1]:
pip install --upgrade tensorflow-gpu==2.0.0-rc0

     |████████████████████████████████| 380.5MB 94kB/s 
     |████████████████████████████████| 71kB 19.6MB/s 
     |████████████████████████████████| 501kB 66.9MB/s 
     |████████████████████████████████| 4.3MB 10.0MB/s 
  Created wheel for opt-einsum: filename=opt_einsum-3.0.1-cp36-none-any.whl size=58500 sha256=eb90d66682c2bc6eed03fa45ee6f8540c2215b7c8561339347227ca59b8902d5
  Stored in directory: /root/.cache/pip/wheels/91/98/8d/10e3d4e04c959597a411b91acd3695e9e2d210e68ce3427aad
Successfully built opt-einsum


In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import numpy as np

tf.__version__

'2.0.0-rc0'

In [3]:
# Get IMDB dataset
imdb, info = tfds.load("imdb_reviews", with_info     = True, 
                                       as_supervised = True)

W0826 14:36:41.410063 140477990496128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/0.1.0. Subsequent calls will reuse this data.


W0826 14:37:17.984565 140477990496128 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [0]:
# Train test split
train_data, test_data = imdb['train'], imdb['test']

training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# str(s.tonumpy()) is needed in Python3 instead of just s.numpy()
for s,l in train_data:
    training_sentences.append(str(s.numpy()))
    training_labels.append(l.numpy())
    
for s,l in test_data:
    testing_sentences.append(str(s.numpy()))
    testing_labels.append(l.numpy())
    
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)


In [6]:
# Sanity check
training_sentences[np.random.randint(len(training_sentences))]

'b\'Relentlessly stupid, no-budget "war picture" made mainly to show off the attributes of the spectacular Eve Meyer--not a bad idea in itself--but that should be an embarrassment to everyone connected with it. Laughable "script", performances that wouldn\\\'t pass muster in an elementary-school Christmas pageant, inept "action" scenes, confused direction by the normally competent documentary director Louis Clyde Stoumen--who is apparently not quite sure if he\\\'s making a comedy, a philosophical treatise on the futility of war or a leering T&A (by early 1960s standards, anyway) travelogue of Eve Meyer\\\'s magnificent body--and a general air of shoddiness and incompetence. Worth seeing in order to watch Eve Meyer strut her stuff, but that\\\'s pretty much it.\''

In [0]:
# Define tokenizer & model params
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [8]:
# Instantiate tokenizer
tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)

# Fit tokenizer
tokenizer.fit_on_texts(training_sentences)

# Get tokens
tokens = tokenizer.word_index

# Print 10 most popular tokens
print(list(tokens.keys())[:10])

['<OOV>', 'the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it']


In [0]:
# Transform sentences (train and test)
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, 
                       maxlen     = max_length, 
                       truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sequences,
                               maxlen     = max_length, 
                               truncating = trunc_type)

In [10]:
# Sanity check - reverse coding
reverse_word_index = dict([(value, key) for (key, value) in tokens.items()])

# print(reverse_word_index)

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(decode_review(padded[1]))
print(training_sentences[1])

b i absolutely loved this movie when i was a kid i cried every time i watched it it wasn't weird to me i totally identified with the characters i would love to see it again and hope i wont be disappointed <OOV> rocks i was really drawn in to the fantasy world and to me the movie was <OOV> i wonder if i ever saw the series and have confused them the acting i thought was strong i loved jack <OOV> he was so dreamy to an 10 year old when i first saw the movie not in 1970 i can still remember the characters vividly the <OOV> was totally believable and i can still <OOV> the evil woods
b"I absolutely LOVED this movie when I was a kid. I cried every time I watched it. It wasn't weird to me. I totally identified with the characters. I would love to see it again (and hope I wont be disappointed!). Pufnstuf rocks!!!! I was really drawn in to the fantasy world. And to me the movie was loooong. I wonder if I ever saw the series and have confused them? The acting I thought was strong. I loved Jack W

In [11]:
# Build a model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 120, 16)           160000    
_________________________________________________________________
flatten (Flatten)            (None, 1920)              0         
_________________________________________________________________
dense (Dense)                (None, 6)                 11526     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Train the model
num_epochs = 10
model.fit(padded, training_labels_final, 
          epochs          = num_epochs, 
          validation_data = (testing_padded, testing_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10


W0826 14:41:38.451199 140477990496128 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_core/python/ops/nn_impl.py:183: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


25000/25000 [==============================] - 8s 323us/sample - loss: 0.4935 - accuracy: 0.7426 - val_loss: 0.4190 - val_accuracy: 0.8089
Epoch 2/10
25000/25000 [==============================] - 6s 255us/sample - loss: 0.2400 - accuracy: 0.9080 - val_loss: 0.4122 - val_accuracy: 0.8195
Epoch 3/10
25000/25000 [==============================] - 6s 256us/sample - loss: 0.0911 - accuracy: 0.9766 - val_loss: 0.5289 - val_accuracy: 0.8020
Epoch 4/10
25000/25000 [==============================] - 7s 268us/sample - loss: 0.0267 - accuracy: 0.9961 - val_loss: 0.6027 - val_accuracy: 0.8066
Epoch 5/10
25000/25000 [==============================] - 6s 260us/sample - loss: 0.0107 - accuracy: 0.9988 - val_loss: 0.6758 - val_accuracy: 0.8054
Epoch 6/10
25000/25000 [==============================] - 6s 253us/sample - loss: 0.0052 - accuracy: 0.9994 - val_loss: 0.7258 - val_accuracy: 0.8063
Epoch 7/10
25000/25000 [==============================] - 6s 252us/sample - loss: 0.0021 - accuracy: 0.9998 - v

In [13]:
# Get embedings for visualization :)
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [0]:
# NOTE: This code is Google Colab-specific

import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')

for word_num in range(1, vocab_size):
    word = reverse_word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
    
out_v.close()
out_m.close()

In [19]:
try:
    from google.colab import files
except ImportError:
    print('Import error')
else:
    print('Downloading...')
    files.download('vecs.tsv')
    files.download('meta.tsv')
    print('Done.')

Downloading...
Done.


Now you can go to https://projector.tensorflow.org/ and visualize IMDB dataset embeddings using the files you just downloded :)